In [6]:
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# load in the data

SB_data = np.load(f"processed_data/od_SB.npy")

print(f"SB data has shape {SB_data.shape}.")

SB data has shape (71, 9).


In [10]:
# train val test split
from sklearn.model_selection import train_test_split

SB_data_train, SB_data_val = train_test_split(SB_data, test_size=0.2, random_state=42)

print(f"SB train data has shape {SB_data_train.shape}.")
print(f"SB val data has shape {SB_data_val.shape}.")

SB train data has shape (56, 9).
SB val data has shape (15, 9).


In [ ]:
# run